# 🍏 Welcome to the **Smart Droplets Hackathon Submission**!

This notebook is here to help you test and submit your solution for the Smart Droplets Hackathon.  
You can use it to validate and evaluate either:

* A trained Reinforcement Learning (RL) agent, or  
* A Conditional agent

Please make sure to follow the instructions in each section carefully.

Good luck — and have fun!

> **Note:** The evaluation environment used in this notebook is the same "validation" environment provided in the hackathon notebook. For the final evaluation, your agent will be tested on an unseen test environment. This test environment is structurally identical to the validation environment, but uses weather data from a different location.


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/WUR-AI/A-scab/blob/hackathon/hackathon/submit_hackathon.ipynb)

# Environment Setup
Before testing your agent, please make sure the environment is properly set up. Use the cell below to import dependencies and initialize the simulation environment.

In [ ]:
# Run this cell to set up the environment

import os
import sys
import subprocess

# Clone the repository
!rm -rf A-scab/
!git clone -b hackathon https://github.com/WUR-AI/A-scab.git

# Change directory
%cd A-scab

# Get the absolute path of the project root *after* changing directory
PROJECT_ROOT = os.getcwd()

# Install poetry if needed.
!pip install -qqq poetry

# Configure poetry to create virtual environments in the project directory
!poetry config virtualenvs.in-project true

# Install project dependencies
# This step may take some time (e.g., 5 minutes)
!poetry install --quiet --all-extras

# These are things you don't need to know about for now :)

# 1. Add the project root to sys.path
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)
    print(f"Added project root {PROJECT_ROOT} to sys.path")

# 2. Get the path to the site-packages directory of the poetry virtual environment
try:
    venv_path_output = subprocess.check_output(['poetry', 'env', 'info', '--path']).decode('utf-8').strip()
    python_version_major_minor = f"python{sys.version_info.major}.{sys.version_info.minor}"
    SITE_PACKAGES_PATH = os.path.join(venv_path_output, 'lib', python_version_major_minor, 'site-packages')
except Exception as e:
    print(f"Could not determine poetry venv site-packages path using 'poetry env info': {e}")
    print("Attempting a common fallback path structure...")
    SITE_PACKAGES_PATH = os.path.abspath(os.path.join('.venv', 'lib', f'python{sys.version_info.major}.{sys.version_info.minor}', 'site-packages'))

# 3. Add the site-packages directory to sys.path
if os.path.exists(SITE_PACKAGES_PATH) and SITE_PACKAGES_PATH not in sys.path:
    sys.path.insert(0, SITE_PACKAGES_PATH)
    print(f"Added {SITE_PACKAGES_PATH} to sys.path")
else:
    print(f"Warning: Could not find site-packages at {SITE_PACKAGES_PATH} or it's already in sys.path.")

import ascab
import gymnasium as gym

# RL Agent
If you have trained a Reinforcement Learning (RL) agent, please use the cells below to test your agent in the simulation environment. This will help validate that your agent behaves as expected and meets the requirements for submission.

## Upload Your Trained Agent
Please run the cell below and use the resulting upload button to select and upload your files.

Do not manually upload files; the code below ensures files are correctly processed and renamed automatically.

Typically, you need to upload:

1.   Your trained model file (e.g., `rl_agent.zip`)
2.   (Optional) Environment normalization file, if you used one during training (e.g., `rl_agent_norm.pkl`)

In [ ]:
from google.colab import files
import os
import shutil

# Upload files
uploaded = files.upload()

# Create /content if it doesn't exist (mostly for local notebook support)
os.makedirs('/content', exist_ok=True)

# Standard filenames
standard_model_name = "rl_agent.zip"
standard_norm_name = "rl_agent_norm.pkl"

def safe_move(src, dst):
    if os.path.exists(dst):
        os.remove(dst)
    shutil.move(src, dst)

# Move and rename uploaded files, allowing overwriting
for filename in uploaded.keys():
    if filename.endswith(".zip") and "norm" not in filename.lower():
        safe_move(filename, f"/content/{standard_model_name}")
        print(f"Renamed and moved '{filename}' → '/content/{standard_model_name}'")
    elif filename.endswith(".pkl") or filename.endswith(".pickle") or "norm" in filename.lower():
        safe_move(filename, f"/content/{standard_norm_name}")
        print(f"Renamed and moved '{filename}' → '/content/{standard_norm_name}'")

## Run you RL agent

Now we’ll load your trained RL agent and run it in the test environment.


*   By default, it assumes you have used the DQN algorithm. If you trained your
*   Similarly, if you customized the observation space, make sure to update the observation_filter to match your model’s expected inputs

In [ ]:
from ascab.train import RLAgent
from ascab.utils.plot import plot_results
from stable_baselines3 import PPO, SAC, TD3, DQN, HER

ascab_val = gym.make('AscabValEnv-Discrete')

algo=DQN
observation_filter=list(ascab_val.observation_space.keys())

rl_agent = RLAgent(
    rl_algorithm=algo,
    observation_filter=list(ascab_val.observation_space.keys()),
    ascab_test=ascab_val,
    path_model='/content/rl_agent',
)
results = rl_agent.run()

## Prepare files for submission

Congratulations! If the final cell executed without errors and you see rewards being reported, your reinforcement learning agent is functioning correctly in the evaluation environment.

You're now ready to submit your results. Please follow these steps:

1. Download your trained agent, and if applicable, the normalization file:  
   - `/content/rl_agent.zip`  
   - `/content/rl_agent_norm.pkl`

2. Download this notebook to include it in your submission.

3. Create a ZIP file containing the downloaded notebook and the trained agent files.

4. Go to the submission section at the end of this notebook


# Conditional Agent

If you have designed a conditional agent, please follow the instructions and code cells below to test and evaluate your model.

The example code provided here defines a simple conditional agent. Replace this example with your own implementation, making sure it runs correctly in the evaluation environment.

In [ ]:
# Conditional agent

from ascab.train import BaseAgent
from ascab.env.env import AScabEnv

# First define the class, not forgetting to SubClass `BaseAgent`
# In general, you want to change the `get_action` method to apply your conditional spraying strategy
class HowMyLocalFarmerSprays(BaseAgent):
    def __init__(
        self,
        ascab: AScabEnv = None,
        render: bool = True,
    ):
        super().__init__(ascab=ascab, render=render)

    def get_action(self, observation: dict = None) -> float:
        # Please include this check for all your conditional agents to constrain it within the "risk period".
        if self.ascab.get_wrapper_attr("info")["InfectionWindow"] and self.ascab.get_wrapper_attr("info")["InfectionWindow"][-1] == 1:
            # The code below means: "If it is forecasted that it will rain in two days, I will spray today".
            if self.ascab.get_wrapper_attr("info")["Forecast_day2_HasRain"] and self.ascab.get_wrapper_attr("info")["Forecast_day2_HasRain"][-1]:
                return 1.0  # the agent sprays this much if is forecasted to rain in two days
        return 0.0

farmer_strategy = HowMyLocalFarmerSprays(ascab_val)

# Use the class method below to run your agent!
results = farmer_strategy.run()

## Prepare files for submission...

Congratulations! If the final cell executed without errors and you see rewards being reported, your conditional agent is functioning correctly in the evaluation environment.

You're now ready to submit your results. Please follow these steps:

1. Download this notebook to include it in your submission.

2. Zip the notebook that you just downloaded

3. Go to the submission section at the end of this notebook

# Submit your files

## Instructions

- Go to this link in huggingface [LINK](https://huggingface.co/spaces/com3dian/hackathon), and there you can upload your submission of either a trained RL agent or a conditional trained agent.
- In that huggingface link, the leaderboard is also displayed!
- You should upload a `.zip` file with these specifications:
    1. The file name should be in the format of `[NAME]_[MODEL_NAME].zip`. So, for example, Hilmy's submission would be `HILMY_AwesomeModel.zip`. It is possible to also do anonymous submissions, if you don't want your name on the leaderboard, which we perfectly understand if that's the case 🙂 In that case, you should also append another underscore, for example `HILMY_AwesomeModel_H4ck3r.zip`. This way will indicate that your nickname will be displayed in the leaderboard.

    2. If it is an RL Agent, the zip file should include all the files needed to re-run the model in this submission Jupyter notebook. That will typically be the saved model and possibly the environment normalization pickle files if applicable. We will assume that all the code to load the required files and run the agent are present in the uploaded evaluation Jupiter notebook.

    2. If it is a Conditional Agent the zip should contain this submission Jupyter notebook file.